In [9]:
import math
from scipy.stats import linregress
from scipy.fft import fftfreq
import pandas as pd
import h5py
import numpy as np

lc = pd.read_csv('AP31396872.csv', usecols=['hjd', 'mag', 'mag_err'])

class EstadisticasLuzEstelar:
    """
    @brief Clase para calcular estadísticas básicas de una serie temporal de magnitudes de luz estelar.

    @param lc: Lista que contiene las magnitudes de luz estelar ('mag').

    @var lista_ordenada: Lista ordenada de las magnitudes.
    @var promedio: Promedio de las magnitudes.
    @var mediana: Mediana de las magnitudes.
    @var varianza: Varianza de las magnitudes.
    """

    def __init__(self, lc):
        self.lc = lc
        self.lista_ordenada = sorted(lc['mag'])
        self.promedio = self.calcular_promedio()
        self.mediana = self.calcular_mediana()
        self.varianza = self.calcular_varianza()

    def calcular_promedio(self):
        """Calcula el promedio de las magnitudes."""
        suma = sum(self.lc['mag'])
        return suma / len(self.lc['mag'])

    def calcular_mediana(self):
        """Calcula la mediana de las magnitudes."""
        cant = len(self.lista_ordenada)
        if cant % 2 == 0:
            return (self.lista_ordenada[cant // 2 - 1] + self.lista_ordenada[cant // 2]) / 2
        else:
            return self.lista_ordenada[cant // 2]

    def calcular_varianza(self):
        """Calcula la varianza de las magnitudes."""
        suma = sum([(i - self.promedio) ** 2 for i in self.lc['mag']])
        return suma / len(self.lc['mag'])

class EstadisticasLuzEstelarExtendido(EstadisticasLuzEstelar):
    """
    @brief Clase que extiende EstadisticasLuzEstelar para calcular estadísticas adicionales.

    @var amplitud: Amplitud de las magnitudes.
    @var kurtosis: Kurtosis de las magnitudes.
    @var autocorrelacion: Coeficiente de autocorrelación de las magnitudes.
    @var asimetria: Asimetría de las magnitudes.
    @var desviacion_estandar: Desviación estándar de las magnitudes.
    @var percentiles: percentiles 20 35 50 65 y 80 de las magnitudes.
    """

    def __init__(self, lc):
        super().__init__(lc)
        self.amplitud = self.calcular_amplitud()
        self.kurtosis = self.calcular_kurtosis()
        self.autocorrelacion = self.calcular_autocorrelacion()
        self.asimetria = self.calcular_asimetria()
        self.desviacion_estandar = self.calcular_desviacion_estandar()

    def calcular_amplitud(self):
        """Calcula la amplitud de las magnitudes."""
        return self.lista_ordenada[-1] - self.mediana

    def calcular_kurtosis(self):
        """Calcula la kurtosis de las magnitudes."""
        suma = sum([(i - self.promedio) ** 4 for i in self.lc['mag']])
        suma2 = sum([(i - self.promedio) ** 2 for i in self.lc['mag']])
        N = len(self.lc['mag'])
        return ((1 / N) * suma) / ((suma2 / N) ** 2) - 3

    def calcular_autocorrelacion(self, lag=1):
        """Calcula el coeficiente de autocorrelación de las magnitudes."""
        lista = self.lc['mag']
        n = len(lista)
        suma = sum([(lista[i] - self.promedio) * (lista[i + lag] - self.promedio) for i in range(n - lag)])
        suma2 = sum([(lista[i] - self.promedio) ** 2 for i in range(n - lag)])
        return suma / suma2

    def calcular_asimetria(self):
        """Calcula la asimetría de las magnitudes."""
        lista = self.lc['mag']
        n = len(lista)
        suma = sum([(lista[i] - self.promedio) ** 3 for i in range(n - 1)])
        suma2 = sum([(lista[i] - self.promedio) ** 2 for i in range(n - 1)])
        return (suma / n) / (suma2 / n) ** (3 / 2)

    def calcular_desviacion_estandar(self):
        """Calcula la desviación estándar de las magnitudes."""
        return math.sqrt(self.varianza)
    
    def percentiles(self):
        """Calcula los percentiles 20 35 50 65 y 80 de las magnitudes"""
        
        lista = self.lc['mag']
    
        p20_index =  int(0.2 * (len(lista) + 1)) - 1
        p20 = lista[p20_index]
    
        p35_index = int(0.35* (len(lista) + 1)) - 1
        p35 = lista[p35_index]
    
        p50_index = int(0.5* (len(lista) + 1)) - 1
        p50 = lista[p50_index]
    
        p65_index = int(0.65* (len(lista) + 1)) - 1
        p65 = lista[p65_index]
    
        p80_index = int(0.8* (len(lista) + 1)) - 1
        p80 = lista[p80_index]
    
        return p20, p35, p50, p65, p80
    

class AnalisisFrecuencias:
    """
    @brief Clase para realizar análisis de frecuencias en una serie temporal de magnitudes de luz estelar.

    @param lc: Arreglo que contiene las magnitudes de luz estelar ('mag') y los tiempos ('hjd').

    @var frecuencia_fundamental: Frecuencia fundamental de la serie temporal.
    """

    def __init__(self, lc):
        self.lc = lc
        self.frecuencia_fundamental = self.calcular_frecuencia_fundamental()

    def calcular_frecuencia_fundamental(self):
        """Calcula la frecuencia fundamental de la serie temporal."""
        tiempo = self.lc['hjd']
        flujo = self.lc['mag']
        N = len(tiempo)
        fs = 1 / (tiempo[1] - tiempo[0])
        frequencies = fftfreq(N, 1 / fs)
        positive_frequencies = frequencies[:N // 2]

        # Calcula el espectro de potencia
        power_spectrum = np.abs(np.fft.fft(flujo)[:N // 2]) ** 2

        # Encuentra la frecuencia fundamental como la frecuencia con mayor potencia
        index_fundamental = np.argmax(power_spectrum[1:]) + 1
        frecuencia_fundamental = positive_frequencies[index_fundamental]

        return frecuencia_fundamental

    def calcular_frecuencias_armonicos(self):
        """Calcula las frecuencias de los armónicos."""
        tiempo = self.lc['hjd']
        num_armonicos = 4
        fs = 1 / (tiempo[1] - tiempo[0])
        frequencies = fftfreq(len(tiempo), 1 / fs)
        positive_frequencies = frequencies[:len(frequencies) // 2]
        return positive_frequencies[1:num_armonicos + 1]

    def calcular_periodo(self):
        """Calcula el período asociado a la frecuencia fundamental."""
        return 1 / self.frecuencia_fundamental

    def calcular_periodograma(self):
        """Calcula el periodograma de la serie temporal."""
        tiempo = self.lc['hjd']
        flujo = self.lc['mag']
        N = len(tiempo)
        dt = tiempo[1] - tiempo[0]
        frecuencias = [1 / (N * dt) * i for i in range(N)]
        potencia = [
            (sum([fl * math.cos(2 * math.pi * f * t) for t, fl in zip(tiempo, flujo)]) ** 2 +
             sum([fl * math.sin(2 * math.pi * f * t) for t, fl in zip(tiempo, flujo)]) ** 2) / N
            for f in frecuencias]
        return frecuencias, potencia
    

d

AttributeError: 'File' object has no attribute 'put'